In [2]:
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import OneHotEncoder

In [3]:
df = pd.read_csv('data/accionsPReprocesadas1.csv')[["Tramit", "Sessio"]]
df.head()

,Tramit,Sessio
0,jNos9Ffia1hbJvPUnfksa3CFdQR/FUICLjKwijU67Z8=,knvOJojBo3o3VEIoZY/viRnmqeM9EH00K0x9rKaxqso=
1,4KE5hVcLhuR5hNmelkoQQkHKbY+pYKWx0izJQGdnvxE=,knvOJojBo3o3VEIoZY/viRnmqeM9EH00K0x9rKaxqso=
2,4KE5hVcLhuR5hNmelkoQQkHKbY+pYKWx0izJQGdnvxE=,knvOJojBo3o3VEIoZY/viRnmqeM9EH00K0x9rKaxqso=
3,miOaT/3KS5U21xkKooUwjngD2sK2hWE3NIofOSjQTnk=,osGe/G7oFf/2275fm0JaeHtB7V+qp3RI+nDuAprxk0o=
4,6ruRKSURKfVwTMFflli1ri+M4TuFoxpIyAjM5Osezlo=,C04XThPdolRf4jadHLW0Z8NPWMIpkoL182E8CqdJyWA=


In [51]:
df = df.sample(frac=0.1)

In [52]:
from collections import defaultdict
new_df = defaultdict(list)

In [53]:
for gname, g_df in (df.groupby("Sessio")):
    if len(g_df) < 2:
        continue
    first_tramit = (g_df.iloc[0]["Tramit"])
    second_tramit = (g_df.iloc[1]["Tramit"])
    new_df["first_tramit"].append(first_tramit)
    new_df["second_tramit"].append(second_tramit)
new_df = pd.DataFrame(new_df)

In [55]:
import numpy as np
onehot = OneHotEncoder(handle_unknown='ignore')
onehot.fit(np.array(list(new_df["first_tramit"]) + list(new_df["second_tramit"]))[:, np.newaxis])

OneHotEncoder(handle_unknown='ignore')

In [57]:
first_tramits = onehot.transform(np.array(list(new_df["first_tramit"]))[:, np.newaxis]).toarray()
second_tramits = onehot.transform(np.array(list(new_df["second_tramit"]))[:, np.newaxis]).toarray()


In [68]:
all_tramits = np.concatenate([first_tramits, second_tramits], axis=1)

In [1]:
from sklearn.neighbors import NearestNeighbors

nn = NearestNeighbors(metric="cosine").fit(all_tramits)

NameError: name 'all_tramits' is not defined

In [115]:
input_ = np.zeros_like(all_tramits[0])
input_[:774 // 2] = all_tramits[0][:774 // 2]
input_[774 // 2:] = 0

In [119]:
output_ = nn.kneighbors(input_[np.newaxis])[1]

In [131]:
_, tramits_idx = np.where(all_tramits[output_][0, :, 387:] ==1)
print(tramits_idx)
onehot.categories_[0][tramits_idx]

[ 56 376 248 110  27]


array(['6dh6weta1jFzRG7on1MoqvG5+0CWST9XX35jbsqt1F4=',
       'xbb+9Y1jPpO/d2uV9KnBrzGQKNc5dfWYBoG306cHFB8=',
       'b1kvVJPFFvRhzfa+YwMUvVrjtbIMTuvVJ2l+ld7eCoY=',
       'EUGUYCD9cTnlwsVdy1T7YiQTRzTHCaySQ5GqBsavzY8=',
       '1szZydSz4dnv9kKdrATkOQZAOQV2PrvnXuXkGTVvitw='], dtype='<U44')

In [135]:
import pickle

with open('onehot.pkl', 'wb') as onehot_file:
    pickle.dump(onehot, onehot_file)
with open("nearest.pkl", 'wb') as nearest_file:
    pickle.dump(nn, nearest_file)
with open("all_tramits.pkl", 'wb') as all_Tramits_file:
    pickle.dump(all_tramits, all_Tramits_file)

In [111]:
df['Sessio']

3043781    ZjP9em8k3EgXxeb97DACkl1o3ONDEKPacUGrbmPz2RA=
1352425    A8UEiVKghwR4aPn+mGWyjlx/qXogsYFm4j2FxBLs4cs=
3038740    mog7lTGJVmW87MNUmLlS3ODRZJiBKGjecXb4d6aywO0=
971200     YgKeEtHz34cUNUb7EYHuiv2o6c6ef3RuPTFOwejUyxY=
899014     V790UKYocFaC+UXr/DAMzxndjcZ6vFCKkLuxWAplvcI=
                               ...                     
2478376    KMLonR32nq2ljAMtxi36pTSedTYyrBNywJ2G4+vN89s=
35275      CCOONiammOD2GdJFIeSC8OoL1R9OMYgJqmoYAjlEMzc=
1794509    ll0e1QSSy96JZBpUtW+DM+s9Pxo2hLvO1PnIoTWo27w=
579463     navjYRT/RvQV3UPMhz76Q+ynZTlmdjatb3eFgvI6Jzk=
3167290    5BkYSQCxgTHB3EabyJGdD55VsLEpXBCtph5jSHqcS4M=
Name: Sessio, Length: 110456, dtype: object

In [109]:
example_first = onehot.transform([["+1JQhTbg4lgyTIo1IfnXsGHt0uVmFQui/PprXOhQCOQ="]]).toarray()
example_entrada = np.zeros((1, 387 * 2))
example_entrada[:, :387] = example_first
example_entrada[:, 387:] = np.nan
output_ = nn.transform(example_entrada)
entrada, sortida = np.where(output_  == 1)
print(onehot.categories_[0][int(entrada[0])], onehot.categories_[0][int(sortida[0])])

+1JQhTbg4lgyTIo1IfnXsGHt0uVmFQui/PprXOhQCOQ= +1JQhTbg4lgyTIo1IfnXsGHt0uVmFQui/PprXOhQCOQ=
